In [1]:
import numpy as np
from collections import namedtuple

In [2]:
def areConformable(mat1, mat2):
    '''
    evaluate conformability of mat1 and mat2
    (i.e. possibility of multiplying mat1 for mat2)
    '''
    if mat1.shape[1] == mat2.shape[0]: return True
    return False

In [2]:
def matrixMultiply(mat1: np.ndarray, mat2: np.ndarray) -> np.ndarray:
    '''
    Perform matrix multiplication:
    1st row of arr1 is multiplied by broadcasting for each col of arr2; 
    broadcasted product columns sums form 1st row of returned matrix and so on
    '''
    if not areConformable(mat1, mat2): raise TypeError('Matrices not conformable')
    
    return np.apply_along_axis( lambda x: np.multiply(x[:, np.newaxis], mat2).sum(axis=0), 
                                axis=1, arr=mat1 )

In [4]:
def matrixProduct(A: np.ndarray, B: np.ndarray):
    '''
    Perform matrix multiplication using for loop
    alternative to matrixMultiply
    '''
    if not areConformable(A, B): raise TypeError('Matrices not conformable')
    
    result = np.zeros((A.shape[0], B.shape[1]))
    
    for i in [*zip(*np.ndenumerate(result))][0]: # i.e. for every tupled index of result:
        result[i[0], i[1]] = (A[i[0], :] * B[:, i[1]]).sum()

    return result

In [5]:
def dotProduct(arr: np.ndarray):
    '''
    Return dot product of an array
    '''
    if arr.ndim > 1: raise TypeError('Array must have 1dim')
    return matMultiply(arr.T, arr)

In [2]:
def couple(x, y): 
    '''
    ufunc that returns tuple of broadcasted values for 2 arrays
    '''
    return x, y
couple = np.frompyfunc(couple, 2, 1)

def bi_index(x):
    '''
    Return an arr equal to index of 2dim passed array
    '''
    row = np.arange(x.shape[0]).reshape((x.shape[0], 1))
    col = np.arange(x.shape[1])
    return np.array(couple(row, col))

def matprod_index(A, B):
    '''
    Return index of matrix product of A and B
    '''
    return couple(
                np.arange(A.shape[0]).reshape(A.shape[0], 1),
                np.arange(B.shape[1])
                )

In [70]:
def as_ufunc(y, x): 
    '''
    ufunc that broadcasts function call to array x
    '''
    return y(x)
as_ufunc = np.frompyfunc(as_ufunc, 2, 1)

In [115]:
def mat_mul(A, B):
    '''
    Equivalent to matrixMultiply(), but uses instantiated ufunc
    '''
    func=lambda x: np.sum(A[x[0], :] * B.T[x[1], :])
    return as_ufunc(func, matprod_index(A, B))

In [6]:
def determinant(mat: np.ndarray):
    '''
    Compute recursively square matrix determinant by Laplace expansion method
    '''
    
    if mat.shape[0] != mat.shape[1]:
        raise TypeError('Matrix must be square')
    
    def helper(x):
        if x.shape[0] <= 2:
            return x[0,0]*x[1,1] - x[0,1]*x[1,0]
        
        cut = np.array( [np.insert(np.delete(x, i, axis=1), 0, x[:, i], axis=1) for i in range(x.shape[0])] )
        
        return sum([i[0,0]*helper(i[1:, 1:]) for i in cut[0::2]]) - sum([j[0,0]*helper(j[1:, 1:]) for j in cut[1::2]])
        
    return helper(mat)

In [7]:
def minors(mat: np.ndarray):
    '''
    Compute matrix of minors of a square matrix.
    For each element of the matrix: 
    1. ignore the values on the current row and column
    2. calculate the determinant of the remaining values 
    '''
    matmin = np.array([])
    
    for i in [*zip(*np.ndenumerate(mat))][0]: # i.e. for every tupled index of mat:
        
        # for each element, find determinant of matrix without row and col of the element
        det = determinant(
                         np.delete(np.delete(mat, i[0], axis=0), i[1], axis=1)
                          )
        matmin = np.append(matmin, det)
    
    return matmin.reshape((mat.shape[0], mat.shape[1]))

In [8]:
def cofactors(mat):
    '''
    Compute matrix of cofactors of a square matrix.
    Applies a mask of - sign to even elements of matrix of minors.
    '''
    mask = np.ones(mat.size).reshape(mat.shape[0], mat.shape[1])
    mask[0::2, 1::2] = -1   # change odd elements of even rows in -1
    mask[1::2, 0::2] = -1   # change even elements of odd rows in -1
    return mask*minors(mat)

In [9]:
def adjugate(matrix):
    '''
    Compute adjugate matrix for square matrix.
    Swap elements of cofactors matrix with respect to diagonal
    '''
    mat = cofactors(matrix)
    
    for i in range(mat.shape[0]):
        for j in range(i):
            mat[i, j], mat[j, i] = mat[j, i], mat[i, j]   
    return mat

Echelon and reduced Echelon form matrices: https://stattrek.com/matrix-algebra/echelon-transform.aspx

In [10]:
def echelon(mat):
    '''
    Calculate recursively row echelon form matrix of mat.
    Echelon form means that:
    1. Each leading entry (!= 0) in every row is 1
    2. Each leading entry is in a column to the right of the leading entry in the previous row
    3. Rows with all zero elements, if any, are below rows having a non-zero element.
    '''
    def helper(matrix):
        mx = np.zeros((matrix.shape[0], matrix.shape[1]))
        mx[1:, 1:] = echelon(matrix[1:, 1:]) 
        return mx
    
    res = np.copy(mat) # fundamental that is a copy, otherwise modifies mat!

    for i in range(res.shape[0]):
        if res[i, 0] != 0:
            
            pivot = np.copy(res[i])
            res = np.delete(res, i, axis=0)
            res = np.insert(res, 0, pivot, axis=0)
            res[0, :] = res[0, :] / res[0, 0]

            for i in range(1, res.shape[0]):
                if res[i, 0] == 0: continue
                res[i, :] = res[i, :] - (res[0, :] * res[i, 0])

            break

    if res.shape[1] < 2:
        return res
    
    mask = np.zeros((res.shape[0], res.shape[1]))
    mask[1:, 1:] = 1
    return np.where(mask == 0, res, helper(res))

In [11]:
def echelonForm(matrix):
    '''
    Equivalent to echelon() but iterative.
    '''
    mat = np.copy(matrix)
    
    # for every column make a copy of submatrix [j:, j:]
    for j in range(mat.shape[1]):
        mx = np.copy(mat[j:, j:])
        
        # and operate on it row transformations
        for i in range(mx.shape[0]):
            if mx[i, 0] != 0:

                pivot = np.copy(mx[i])
                mx = np.delete(mx, i, axis=0)
                mx = np.insert(mx, 0, pivot, axis=0)
                mx[0, :] = mx[0, :] / mx[0, 0]

                for i in range(1, mx.shape[0]):
                    if mx[i, 0] == 0: continue
                    mx[i, :] = mx[i, :] - (mx[0, :] * mx[i, 0])
                
                # save row transformations in correspondent submatrix and come back to col for loop
                mat[j:, j:] = mx
                break
    return mat

In [12]:
def isEchelon(mat):
    '''
    Evaluate if mat is in echelon form.
    '''
    # for every row: proceeding from left to right,
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            if mat[i, j] == 0:   continue      # if value is 0 go on
            elif mat[i, j] == 1: break         # if it's 1 go on to next row (leading entry)
            else:                return False  # if it's a different value mat is not in echelon form
    return True

In [13]:
def isReduced(mat):
    '''
    Evaluate if matrix is in echelon reduced form, i.e. if every laeding entry is unique nonzero value
    in its column, and return positions of leading entries that does not satisfy this condition.
    '''
    if not isEchelon(mat): raise TypeError('Matrix not in echelon form')
    
    Res    = namedtuple('Res', ['reduced', 'reducible'])
    result = Res(True, [])
    
    # for every row
    for i in range(mat.shape[0]):
        # search leading entry. When found:
        for j in range(mat.shape[1]):
            if mat[i, j] == 1:
                # if there are other nonzero value in column, return False
                if mat[:, j].sum() != 1:
                    result.reducible.append((i, j))
                # if not, go on with next row
                break
                
    if not result.reducible == []: 
        return result._replace(reduced=False)
    return result

In [14]:
def reduce(mx):
    '''
    Calculates reduced echelon form matrix.
    For each reducible column, add a multiple of leading entry's row to rows 
    where value is nonzero , so that the nonzero value becomes zero.
    Calls echelon for mx not in echelon form.
    '''
    mat = np.copy(mx)
    if not isEchelon(mat): mat = echelonForm(mat)
    
    for i, j in isReduced(mat).reducible:
        for k in range(i):
            if mat[k, j] != 0:
                mat[k, :] = mat[k, :] - (mat[i, :]*mat[k, j])
    return mat  

Rank of a matrix: https://stattrek.com/matrix-algebra/matrix-rank.aspx

In [15]:
def rank(mat):
    '''
    Return rank of a matrix.
    It is the n of linearly independent rows or cols.
    '''
    mx = echelonForm(mat)
    
    # beginning rank:
    rank = mx.shape[0]
    
    # for every row of only zeros, subtract 1 from rank
    for i in (mx == 0).sum(axis=1):
        if i == mx.shape[1]: rank -= 1
            
    return rank

In [16]:
def isFullRank(mat):
    '''
    Return True if rank is equal to minimum between rows and cols n.
    '''
    return min(mat.shape[0], mat.shape[1]) == rank(mat)

In [17]:
def isSingular(mat):
    '''
    Evaluate if a square matrix is singular (i.e. non invertible).
    Determinant of a nonsingular matrix is different from 0, determinant of a singular matrix is 0.
    If a matrix is nonsingular, it can be canceled from both sides of an equation, 
    provided it appears on the left (or right) on both sides.
    '''
    if mat.shape[0] != mat.shape[1]:
        return TypeError('Matrix not square')
    
    return not isFullRank(mat)

In [18]:
def isSing(mat):
    '''
    Equivalent to isSingular but using determinant instead of rank.
    Singular matrices have determinant 0, nonsingular != 0.
    '''
    if mat.shape[0] != mat.shape[1]:
        return TypeError('Matrix not square')
    
    return determinant(A) == 0

Inverse of a matrix using Minors, Cofactors, Adjugate and Determinant: https://www.mathsisfun.com/algebra/matrix-inverse-minors-cofactors-adjugate.html

In [19]:
def inverse(mat):
    '''
    Compute inverse matrix of square matrix.
    '''
    if isSingular(mat): return TypeError('Matrix is not nonsingular')
    return adjugate(mat) / determinant(mat)

Gauss-Jordan inverting method: https://www.mathsisfun.com/algebra/matrix-inverse-row-operations-gauss-jordan.html

In [20]:
def invert(matrix):
    '''
    Equivalent to inverse(), but uses Gauss-Jordan method. 
    Identity matrix is added along rows of matrix, then rows are multiplied and added to
    obtain identity matrix on the left, leaving inverse matrix where identity matrix was originally appended.
    '''
    if isSingular(matrix): return TypeError('Matrix is not nonsingular')

    mx = np.concatenate((matrix, np.identity(matrix.shape[0])), axis=1)

    for i in range(matrix.shape[0]):
        mx[i, :] = mx[i, :] / mx[i, i]

        for j in [x for x in range(matrix.shape[0]) if x != i]:
            coeff = -mx[j, i]
            add = mx[i, :]*coeff
            mx[j, :] = mx[j, :] + add
            
    return mx[:, matrix.shape[1] : matrix.shape[1]*2]

In [21]:
def diagonal(arr: np.ndarray):
    '''
    Return diagonal matrix whose diagonal is input array
    '''
    if arr.ndim > 1:
        raise TypeError('Array must have only 1dim')
    return np.identity(arr.shape[0]) * arr